In [1]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch, onecold, crossentropy, throttle
using Flux: @epochs
using Statistics
using MLDatasets
using Serialization: serialize, deserialize
using PyCall
using ImageView
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra

In [2]:
x_train, y_train = SVHN2.traindata()
x_test, y_test = SVHN2.testdata()

(FixedPointNumbers.N0f8[0.149N0f8 0.153N0f8 … 0.196N0f8 0.188N0f8; 0.153N0f8 0.153N0f8 … 0.2N0f8 0.188N0f8; … ; 0.165N0f8 0.169N0f8 … 0.176N0f8 0.173N0f8; 0.153N0f8 0.153N0f8 … 0.165N0f8 0.165N0f8]

FixedPointNumbers.N0f8[0.404N0f8 0.408N0f8 … 0.459N0f8 0.451N0f8; 0.408N0f8 0.408N0f8 … 0.463N0f8 0.451N0f8; … ; 0.404N0f8 0.396N0f8 … 0.455N0f8 0.451N0f8; 0.38N0f8 0.38N0f8 … 0.443N0f8 0.443N0f8]

FixedPointNumbers.N0f8[0.235N0f8 0.239N0f8 … 0.298N0f8 0.29N0f8; 0.239N0f8 0.239N0f8 … 0.302N0f8 0.29N0f8; … ; 0.243N0f8 0.247N0f8 … 0.282N0f8 0.278N0f8; 0.224N0f8 0.224N0f8 … 0.271N0f8 0.278N0f8]

FixedPointNumbers.N0f8[0.506N0f8 0.525N0f8 … 0.541N0f8 0.514N0f8; 0.498N0f8 0.522N0f8 … 0.51N0f8 0.478N0f8; … ; 0.482N0f8 0.494N0f8 … 0.396N0f8 0.435N0f8; 0.482N0f8 0.49N0f8 … 0.439N0f8 0.482N0f8]

FixedPointNumbers.N0f8[0.557N0f8 0.588N0f8 … 0.596N0f8 0.569N0f8; 0.561N0f8 0.584N0f8 … 0.565N0f8 0.533N0f8; … ; 0.525N0f8 0.537N0f8 … 0.42N0f8 0.463N0f8; 0.529N0f8 0.537N0f8 … 0.463N0f8 0.51N0f8]

FixedPoin

In [3]:
epochs = 100;
batch_size = 64;

In [4]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[:,:,1, 1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] .= Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 1:10)
    return (X_batch, Y_batch)
end

make_minibatch (generic function with 1 method)

In [5]:
mb_idxs = partition(1:size(x_train)[4], batch_size)
train_set = [make_minibatch(x_train, y_train, i) for i in mb_idxs];
mb_idxs_test = partition(1:size(x_test)[4], batch_size)
test_set = make_minibatch(x_test, y_test, 1:size(x_test)[4]);

In [6]:
function loss(x, y)
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x)) #szum losowy
    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
opt = ADAM(0.001)

ADAM(0.001, (0.9, 0.999), IdDict{Any, Any}())

In [7]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Dropout(0.2),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Dropout(0.2),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>64, pad=(1,1), relu),
    x -> reshape(x, :, size(x, 4)),
    Dense(1024, 256, relu),
    Dense(256, 64, elu),
    Dropout(0.5),
    Dense(64,10),
    softmax
);

In [8]:
model(test_set[1])

10×26032 Matrix{Float32}:
 0.0973738  0.0973054  0.0973054  …  0.0882875  0.0886673  0.0891121
 0.100664   0.10068    0.10068       0.102526   0.102462   0.102385
 0.100147   0.100151   0.100151      0.100557   0.100543   0.100526
 0.101309   0.101342   0.101342      0.105496   0.105332   0.105138
 0.0945639  0.0944251  0.0944251     0.0773232  0.077998   0.0787933
 0.102182   0.10224    0.10224    …  0.109974   0.109639   0.109248
 0.101173   0.101203   0.101203      0.104949   0.1048     0.104624
 0.101665   0.101709   0.101709      0.107363   0.107127   0.106849
 0.0994742  0.0994596  0.0994596     0.0971447  0.0972578  0.0973886
 0.101448   0.101485   0.101485      0.106379   0.106174   0.105935

In [9]:
@time accuracy(test_set[:][1],test_set[:][2])

 25.184027 seconds (1.15 M allocations: 6.138 GiB, 9.33% gc time, 4.02% compilation time)


0.07594499078057775

In [31]:
Flux.train!(loss, params(model), train_set, opt)

In [10]:
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch = 1:epochs
    global best_acc, last_improvement
    Flux.train!(loss, params(model), train_set, opt)
    acc = accuracy(test_set[1],test_set[2])
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))
    if acc >= 0.96
        @info(" -> Early-exiting: We reached our target accuracy of 96%")
        break
    end
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to SVHN_final_project.bson")
        BSON.@save "SVHN_final_project.bson" model epoch acc
        best_acc = acc
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end

┌ Info: Beginning training loop...
└ @ Main In[10]:1
┌ Info: [1]: Test accuracy: 0.1959
└ @ Main In[10]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_final_project.bson
└ @ Main In[10]:14
┌ Info: [2]: Test accuracy: 0.1959
└ @ Main In[10]:8
┌ Info:  -> New best accuracy! Saving model out to SVHN_final_project.bson
└ @ Main In[10]:14


LoadError: InterruptException:

In [11]:
BSON.@load "SVHN_final_project.bson" model epoch acc